In [1]:
import pandas as pd
import os

In [2]:
def Get_TekMap_Paths(folder):
    paths = []
    for folder, dirs, files in os.walk(folder):
        for file in files:
            fullpath = os.path.join(folder, file)
            with open(fullpath, 'r') as f:
                for line in f:
                    if line[:7].lower() != 'rowdata':
                        paths += [fullpath]
                        break
    return paths

def TekMap_to_DataFrame(path):
    with open(path) as f:
        lines = f.readlines()
    header = []
    tekmap = []
    # parsing through lines to pull out header info into header list, then the map info into the tekmap list
    for x in lines:
        if x[:7].lower() != 'rowdata':
            header += [x]
        else:
            # appending the row, with the ROWDATA: and \n removed, then split into an array,
            # then reversed horizontally to match the row/col reference
            tekmap += [x[8:-2].split(' ')[::-1]]

    # making an emply list to fill with row, column, and results info from the wafer map
    maplist = []

    # parsing through tekmap to get row, column, and results in the new list
    for n,x in enumerate(tekmap):
        for m,y in enumerate(x):
            maplist += [[n+1,m+1,y]]

    # creating a new pandas dataframe using the row, column, and result list
    df = pd.DataFrame(data=maplist, columns=['row','column','result'])

    # adding the header information as columns, in reverse order since the columns are being added to the front of the dataframe each time
    for x in header[::-1]:
        df.insert(0,x.split(':')[0],x.split(':')[1][:-1])
        
    return df[df['result'] != '__']

def Append_TekMap_DataFrames(folder):
    # getting the file paths to parse through
    paths = Get_TekMap_Paths(folder)
    # creating dummy variable to use to create the first dataframe
    first = 1
    for x in paths:
        if first == 1:
            # making the first dataframe, setting first to 0
            df_maps = TekMap_to_DataFrame(x)
            first = 0
        else:
            # appending the rest of the dataframes
            df_maps = df_maps.append(TekMap_to_DataFrame(x))
    return df_maps

In [3]:
folder = r'C:\path\to\folder'

df = Append_TekMap_DataFrames(folder)

In [5]:
df.to_csv(folder+'\\'+'mapTable.csv')

In [ ]:
for n,x in enumerate(tekmap):
    for m,y in enumerate(x):
        maplist += [[n,m,y]]

# creating a new pandas dataframe using the row, column, and result list
df = pd.DataFrame(data=maplist, columns=['row','column','result'])

# adding the header information as columns, in reverse order since the columns are being added to the front of the dataframe each time
for x in header[::-1]:
    df.insert(0,x.split(':')[0],x.split(':')[1][:-1])